In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

In [0]:
!apt-get install build-essential cmake -y
!apt-get install libgtk-3-dev -y
!apt-get install libboost-all-dev -y
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
!pip install dlib

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p colab_data
!google-drive-ocamlfuse -o nonempty colab_data

In [0]:
# !wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
# !tar zxvf lfw.tgz
# !rm lfw.tgz
# !mv -rf lfw cbdatas

In [0]:
# !mkdir -p wf

# !wget http://ww3.sinaimg.cn/large/00711cMmly1folvw6u79nj306o08w74k.jpg
# !wget http://ww2.sinaimg.cn/large/00711cMmly1folvw6voqbj306o08wjsd.jpg
# !wget http://ww4.sinaimg.cn/large/00711cMmly1folvw7oyumj30ga0kfq4t.jpg
# !wget http://ww4.sinaimg.cn/large/00711cMmly1folvw88hozj30ku0t6wq3.jpg
# !wget http://ww3.sinaimg.cn/large/00711cMmly1folvw9nom5j30pu106tn5.jpg
# !wget http://ww1.sinaimg.cn/large/00711cMmly1folvwbrzvpj30ku0t64mc.jpg
# !wget http://ww1.sinaimg.cn/large/00711cMmly1folvwcf5kej30pu1067wh.jpg

# !mv *.jpg wf
# !mv -r wf colab_data/cbdatas

---

In [0]:
import tensorflow as tf
import cv2
import numpy as np
import os
import random
import sys
import dlib
from sklearn.model_selection import train_test_split
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片
%matplotlib inline

---

In [0]:
#使用dlib自带的frontal_face_detector作为我们的特征提取器
detector = dlib.get_frontal_face_detector()

In [0]:
# 截取照片中的脸保存为图像
def get_face(input_dir, output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
  index = 1
  for (path, dirnames, filenames) in os.walk(input_dir):

    for filename in filenames:
      print filename
      if filename.endswith('.jpg'):
        print 'Being processed picture %s' % index
        img_path = path+'/'+filename
        img = cv2.imread(img_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转为灰度图片
        # 使用detector进行人脸检测 dets为返回的结果
        dets = detector(gray_img, 1)

        #使用enumerate 函数遍历序列中的元素以及它们的下标
        #下标i即为人脸序号
        #left：人脸左边距离图片左边界的距离 ；right：人脸右边距离图片左边界的距离 
        #top：人脸上边距离图片上边界的距离 ；bottom：人脸下边距离图片上边界的距离
        for i, d in enumerate(dets):
          x1 = d.top() if d.top() > 0 else 0
          y1 = d.bottom() if d.bottom() > 0 else 0
          x2 = d.left() if d.left() > 0 else 0
          y2 = d.right() if d.right() > 0 else 0

          # img[y:y+h,x:x+w]
          face = img[x1:y1, x2:y2]
          face = cv2.resize(face, (size, size))
  #         plt.imshow(face)
          cv2.imwrite(output_dir+'/'+str(index)+'.jpg', face)
          index += 1   
  #   if index > 2:
  #     break 
  plt.show()
  print 'done'


In [0]:
size = 64

input_dir = 'colab_data/cbdatas/wf'
output_dir = 'colab_data/cbdatas/my'
get_face(input_dir, output_dir)

input_dir = 'colab_data/cbdatas/fw'
output_dir = 'colab_data/cbdatas/others'
get_face(input_dir, output_dir)

---

In [0]:
my_faces_path = 'colab_data/cbdatas/my'
other_faces_path = 'colab_data/cbdatas/others'
size = 64

In [0]:
def getPaddingSize(img):
    h, w, _ = img.shape
    top, bottom, left, right = (0,0,0,0)
    longest = max(h, w)

    if w < longest:
        tmp = longest - w
        # //表示整除符号
        left = tmp // 2
        right = tmp - left
    elif h < longest:
        tmp = longest - h
        top = tmp // 2
        bottom = tmp - top
    else:
        pass
    return top, bottom, left, right

In [0]:
def readData(path , h=size, w=size):
    for filename in os.listdir(path):
        if filename.endswith('.jpg'):
            filename = path + '/' + filename

            img = cv2.imread(filename)

            top,bottom,left,right = getPaddingSize(img)
            # 将图片放大， 扩充图片边缘部分
            img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
            img = cv2.resize(img, (h, w))

            imgs.append(img)
            labs.append(path)

In [26]:
imgs = []
labs = []
readData(my_faces_path)
print len(labs)
readData(other_faces_path)
print len(labs)

7
143


In [27]:
# 将图片数据与标签转换成数组
imgs = np.array(imgs)
labs = np.array([[0,1] if lab == my_faces_path else [1,0] for lab in labs])

# 随机划分测试集与训练集
train_x,test_x,train_y,test_y = train_test_split(imgs, labs, test_size=0.2, random_state=0)

# 参数：图片数据的总数，图片的高、宽、通道
train_x = train_x.reshape(train_x.shape[0], size, size, 3)
test_x = test_x.reshape(test_x.shape[0], size, size, 3)

# 将数据转换成小于1的数
train_x = train_x.astype('float32')/255.0
test_x = test_x.astype('float32')/255.0
print('train size:%s, test size:%s' % (len(train_x), len(test_x)))

train size:114, test size:29


In [0]:
# 图片块，每次取100张图片
batch_size = 10
num_batch = len(train_x) // batch_size
x = tf.placeholder(tf.float32, [None, size, size, 3])
y_ = tf.placeholder(tf.float32, [None, 2])
keep_prob_5 = tf.placeholder(tf.float32)
keep_prob_75 = tf.placeholder(tf.float32)

In [0]:
def weightVariable(shape):
    init = tf.random_normal(shape, stddev=0.01)
    return tf.Variable(init)

def biasVariable(shape):
    init = tf.random_normal(shape)
    return tf.Variable(init)
  
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')  # N,H,W,C

def maxPool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')  # ksize 池化窗口大小

def dropout(x, keep):
    return tf.nn.dropout(x, keep)
  
def cnn_layer():
    '''第一层'''
    W1 = weightVariable([3, 3, 3, 32])# 卷积核大小(3,3)， 输入通道(3)， 输出通道(32)
    b1 = biasVariable([32])
    # 卷积
    conv1 = tf.nn.relu(conv2d(x, W1) + b1)
    # 池化
    pool1 = maxPool(conv1)
    # 减少过拟合，随机让某些权重不更新
    drop1 = dropout(pool1, keep_prob_5)
    
    '''第二层'''
    W2 = weightVariable([3,3,32,64])
    b2 = biasVariable([64])
    conv2 = tf.nn.relu(conv2d(drop1, W2) + b2)
    pool2 = maxPool(conv2)
    drop2 = dropout(pool2, keep_prob_5)

    '''第三层'''
    W3 = weightVariable([3,3,64,64])
    b3 = biasVariable([64])
    conv3 = tf.nn.relu(conv2d(drop2, W3) + b3)
    pool3 = maxPool(conv3)
    drop3 = dropout(pool3, keep_prob_5)
    
    '''全连接层'''
    Wf = weightVariable([8 * 16 * 32,  512])
    bf = biasVariable([512])
    drop3_flat = tf.reshape(drop3, [-1, 8 * 16 * 32])
    dense = tf.nn.relu(tf.matmul(drop3_flat, Wf) + bf)
    dropf = dropout(dense, keep_prob_75)
    
    '''输出层'''
    Wout = weightVariable([512, 2])
    bout = weightVariable([2])
    out = tf.add(tf.matmul(dropf, Wout), bout)
    return out

In [0]:
def cnn_train():
  out = cnn_layer()
  
  # 交叉熵损失
  cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=y_))
  
  # 优化
  train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
  
  # 精确度
  accuracy = tf.reduce_mean(tf.cast(
      tf.equal(tf.argmax(out, 1), tf.argmax(y_, 1)), 
      tf.float32))  # cast 转换类型
  
  # 将loss与accuracy保存以供tensorboard使用
  tf.summary.scalar('loss', cross_entropy)
  tf.summary.scalar('accuracy', accuracy)
  merged_summary_op = tf.summary.merge_all()
  
  # 数据保存器的初始化
  saver = tf.train.Saver()
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 百脸懵逼
    summary_writer = tf.summary.FileWriter('./tmp', graph=tf.get_default_graph())
    
    for n in range(10):
      # 每次取128(batch_size)张图片
      for i in range(num_batch):
        batch_x = train_x[i*batch_size : (i+1)*batch_size]
        batch_y = train_y[i*batch_size : (i+1)*batch_size]
        # 开始训练数据，同时训练三个变量，返回三个数据
        _,loss,summary = sess.run(
            [train_step, cross_entropy, merged_summary_op], 
            feed_dict={x:batch_x,y_:batch_y, keep_prob_5:0.5,keep_prob_75:0.75})
        
        summary_writer.add_summary(summary, n*num_batch+i)
        
        # 打印损失
        if (n*num_batch+i) % 30 == 0:
          # 获取测试数据的准确率
          acc = accuracy.eval(
              {x:test_x, y_:test_y, keep_prob_5:1.0, keep_prob_75:1.0})
          print(n*num_batch+i, acc)
          
          # 准确率大于0.98时保存并退出
          if acc > 0.98 and n > 2:
            saver.save(sess, 'colab_data/cbdatas/train_faces.model', global_step=n*num_batch+i)
            
    print('accuracy less 0.98, exited!')

In [41]:
cnn_train()

(0, 1.0)
(30, 1.0)
(60, 1.0)
(90, 1.0)
accuracy less 0.98, exited!


### 使用模型进行识别任务

In [0]:
output = cnnLayer()  
predict = tf.argmax(output, 1)  

saver = tf.train.Saver()  
sess = tf.Session()  
saver.restore(sess, tf.train.latest_checkpoint('.'))  

def is_my_face(image):  
    res = sess.run(predict, feed_dict={x: [image/255.0], keep_prob_5:1.0, keep_prob_75: 1.0})  
    if res[0] == 1:  
        return True  
    else:  
        return False  

#使用dlib自带的frontal_face_detector作为我们的特征提取器
detector = dlib.get_frontal_face_detector()

cam = cv2.VideoCapture(0)  

while True:  
    _, img = cam.read()  
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dets = detector(gray_image, 1)
    if not len(dets):
        #print('Can`t get face.')
        cv2.imshow('img', img)
        key = cv2.waitKey(30) & 0xff  
        if key == 27:
            sys.exit(0)

    for i, d in enumerate(dets):
        x1 = d.top() if d.top() > 0 else 0
        y1 = d.bottom() if d.bottom() > 0 else 0
        x2 = d.left() if d.left() > 0 else 0
        y2 = d.right() if d.right() > 0 else 0
        face = img[x1:y1,x2:y2]
        # 调整图片的尺寸
        face = cv2.resize(face, (size,size))
        print('Is this my face? %s' % is_my_face(face))

        cv2.rectangle(img, (x2,x1),(y2,y1), (255,0,0),3)
        cv2.imshow('image',img)
        key = cv2.waitKey(30) & 0xff
        if key == 27:
            sys.exit(0)

sess.close()